# Bangsamoro Earthquake Data Visualization
## Links
1. https://towardsdatascience.com/how-to-create-an-interactive-geographic-map-using-python-and-bokeh-12981ca0b567

## Instructions(Local)
0. Make sure packages required are installed : `pip install -r requirements.txt`
1. Running this notebook is not needed. We can go directly to command line and run `bokeh serve path/to/this/ipynb --show`
2. Open localhost:5006/earthquake on your browser or whatever link is provided after step 1
3. Wait for a while as bokeh is quite slow(possibly depend on your machine)

## Todo
1. Include non BARMM Geodata as the visualization has a lot of white space, but gray out this non BARMM as this is not part of the project
2. Fix hover, it is not working when changing to medium risk earthquake data
3. Deploy to heroku
4. See if PWA/Offline App possible

In [ ]:
import pandas as pd
import numpy as np
import math

import geopandas as gpd
import shapely.geometry as gm
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

# Geo Data Preparation

In [ ]:
highrisk = pd.read_csv('data/high_risk0.csv')
medrisk = pd.read_csv('data/midrisk.csv')
medrisk_sea = pd.read_csv('data/medium.csv')

In [ ]:
highrisk = gpd.GeoDataFrame(highrisk.drop(['latitude','longitude'],axis=1), crs={'init': 'epsg:4326'} ,geometry = [gm.Point(latlong) for latlong in zip(highrisk.longitude, highrisk.latitude)])
medrisk = gpd.GeoDataFrame(medrisk.drop(['latitude','longitude'],axis=1), crs={'init': 'epsg:4326'} ,geometry = [gm.Point(latlong) for latlong in zip(medrisk.longitude, medrisk.latitude)])
medrisk_sea = gpd.GeoDataFrame(medrisk_sea.drop(['latitude','longitude'],axis=1), crs={'init': 'epsg:4326'} ,geometry = [gm.Point(latlong) for latlong in zip(medrisk_sea.longitude, medrisk_sea.latitude)])

In [ ]:
bm = gpd.read_file('mapdata/AdministrativeBoundariesBARMMMunicipalities20190206PSA2016.shp')

In [ ]:
bm = bm.to_crs(epsg=4236)

# Initial Visualizations

In [ ]:
x = bm.plot(color='lightblue', edgecolor='gray', figsize=(30,30))
y = bm.plot(color='lightblue', edgecolor='gray', figsize=(30,30))
z = bm.plot(color='lightblue', edgecolor='gray', figsize=(30,30))

In [ ]:
highrisk.plot(ax=x, color='red')

In [ ]:
medrisk.plot(ax=x, color='orange')
medrisk_sea.plot(ax=x, color='green')

In [ ]:
x.get_figure()

# Geo Data Preprocessing

In [ ]:
lenbm = len(bm)
lenhr = len(highrisk)
lenmdr = len(medrisk)#combine all medrisk  /
lenmdr2 = len(medrisk_sea)
bm_range = range(lenbm)
highrisk_range = range(lenhr)
medrisk_range = range(lenmdr)
medrisksea_range = range(lenmdr2)

In [ ]:
bangsamoro_risk = {'high':[],'med':[], 'med_sea':[]}
#max_geo_distance = 180 #assuming that max distance of geoobjects is 180'.''
for x in bm_range:
    municipal_highrisk = 0
    municipal_medrisk = 0
    municipal_medrisksea = 0
    for y in highrisk_range:
        municipal_highrisk += bm.geometry[x].centroid.distance(highrisk.geometry[y])
    for z in medrisk_range:
        municipal_medrisk += bm.geometry[x].centroid.distance(medrisk.geometry[z])
    for v in medrisksea_range:
        municipal_medrisksea += bm.geometry[x].centroid.distance(medrisk_sea.geometry[v])
    bangsamoro_risk['high'].append(1/(municipal_highrisk)**2)
    bangsamoro_risk['med'].append(1/(municipal_medrisk)**2)
    bangsamoro_risk['med_sea'].append(1/(municipal_medrisksea)**2)

In [ ]:
#add normalized to geodata for choropleth
bm['high'] = bangsamoro_risk['high']
bm['high'] = bm['high']/bm['high'].max()
bm['med'] = bangsamoro_risk['med']
bm['med'] = bm['med']/bm['med'].max()
bm['med_sea'] = bangsamoro_risk['med_sea']
bm['med_sea'] = bm['med_sea']/bm['med_sea'].max()

In [ ]:
population = pd.read_csv('mapdata/municipal_population.csv')

In [ ]:
bm = bm.merge(population[['POPULATION','PSGC_CITY/MUNI']], left_on='Mun_Code', right_on='PSGC_CITY/MUNI').drop('PSGC_CITY/MUNI', axis=1)

# Bokeh Visualization

In [ ]:
def update_plot(attr,old,new):
    severity=select.value
    new_data = json_data(severity)
    #input_field
    geosource.geojson = new_data
    hover = HoverTool(tooltips = [('Municipality','@Mun_Name'),(severity+' risk','@'+severity)])
    p = make_plot(severity)
    layout=column(p,widgetbox(select))
    curdoc().clear()
    curdoc().add_root(layout)

In [ ]:
def json_data(severity):
    if severity == 'high':
        return json.dumps(json.loads(bm.drop(['med', 'med_sea'], axis=1).to_json()))
    if severity == 'med':
        return json.dumps(json.loads(bm.drop(['high', 'med_sea'], axis=1).to_json()))
    return json.dumps(json.loads(bm.drop(['high', 'med'],axis=1).to_json()))    

In [ ]:
def make_plot(severity):
    color_mapper = LinearColorMapper(palette=palette,low=bm[severity].min(), high=bm[severity].max())
    #format_tick = NumeralTickFormatter(format='0.0')
    color_bar = ColorBar(color_mapper=color_mapper, location=(0,0))
    p = figure(title='Title', plot_height = 650, plot_width = 850, toolbar_location = None)
    #p = figure()
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False
    p.patches('xs', 'ys', source=geosource, fill_color = {'field' : severity, 'transform' : color_mapper}, line_color = 'gray', line_width = 0.25, fill_alpha = 1)
    p.add_layout(color_bar, 'right')
    p.add_tools(hover)
    return p

In [ ]:
severity = 'high'
geosource = GeoJSONDataSource(geojson = json_data(severity))

In [ ]:
palette = brewer['Blues'][8]
palette = palette[::-1]
hover = HoverTool(tooltips = [('Municipality','@Mun_Name'),(severity+' risk','@'+severity),('Population','@POPULATION')])
p = make_plot(severity)
select = Select(title='Select Severity:', value=severity, options=['high','med','med_sea'])
#maybe we could add toggle for epicenters and evacuation centers
select.on_change('value', update_plot)
layout=column(p, widgetbox(select))
curdoc().add_root(layout)

In [ ]:
#output_notebook() #comment out when deploying to heroku

In [ ]:
#p= figure() #comment out when deploying to heroku

In [ ]:
#show(p) #comment out when deploying to heroku

In [ ]:
import chart_studio.plotly as py_cloud
import plotly.offline as py

In [ ]:
layout = dict(
    hovermode = 'closest',
    xaxis = dict(
        autorange = False,
        range = [-125, -65],
        showgrid = False,
        zeroline = False,
        fixedrange = True
    ),
    yaxis = dict(
        autorange = False,
        range = [25, 49],
        showgrid = False,
        zeroline = False,
        fixedrange = True
    ),
    margin = dict(
        t=20,
        b=20,
        r=20,
        l=20
    ),
    width = 1100,
    height = 650,
    dragmode = 'select'
)

In [ ]:
df = pd.DataFrame(bm)

In [ ]:
plot_data = []
for index,row in df.iterrows():
    if df['geometry'][index].type == 'Polygon':
        x,y = row.geometry.exterior.xy
        c_x,c_y = row.geometry.centroid.xy
    elif df['geometry'][index].type == 'MultiPolygon':
        x = [poly.exterior.xy[0] for poly in df['geometry'][index]]
        y = [poly.exterior.xy[1] for poly in df['geometry'][index]]
        x_c = [poly.centroid.xy[0] for poly in df['geometry'][index]]
        y_c = [poly.centroid.xy[1] for poly in df['geometry'][index]]        
    else: 
        print('stop')
    county_outline = dict(
            type = 'scatter',
            showlegend = False,
            legendgroup = "shapes",
            line = dict(color='black', width=1),
            x=x,
            y=y,
            fill='toself',
            fillcolor = 'purple',
            hoverinfo='none'
    )
    hover_point = dict(
            type = 'scatter',
            showlegend = False,
            legendgroup = "centroids",
            name = row.Mun_Name,
            marker = dict(size=2),
            x=c_x,
            y=c_y,
            fill='toself',
            fillcolor = 'purple'            
    )
    plot_data.append(county_outline)
    plot_data.append(hover_point)

In [ ]:
len(plot_data)

In [ ]:
fig = dict(data=plot_data, layout=layout)
py.plot(fig, filename='bangs.html')

In [ ]:
bm

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString
import plotly.graph_objs as go
from choropleth_colors import *

In [ ]:
def shapefile_to_geojson(gdf, index_list, level = 1, tolerance=0.025): 
    # gdf - geopandas dataframe containing the geometry column and values to be mapped to a colorscale
    # index_list - a sublist of list(gdf.index)  or gdf.index  for all data
    # level - int that gives the level in the shapefile
    # tolerance - float parameter to set the Polygon/MultiPolygon degree of simplification
    
    # returns a geojson type dict 
   
    geo_names = list(gdf[f'Mun_Name'])
    geojson = {'type': 'FeatureCollection', 'features': []}
    for index in index_list:
        geo = gdf['geometry'][index].simplify(tolerance)
    
        if isinstance(geo.boundary, LineString):
            gtype = 'Polygon'
            bcoords = np.dstack(geo.boundary.coords.xy).tolist()
    
        elif isinstance(geo.boundary, MultiLineString):
            gtype = 'MultiPolygon'
            bcoords = []
            for b in geo.boundary:
                x, y = b.coords.xy
                coords = np.dstack((x,y)).tolist() 
                bcoords.append(coords) 
        else: pass
        
        
       
        feature = {'type': 'Feature', 
                   'id' : index,
                   'properties': {'name': geo_names[index]},
                   'geometry': {'type': gtype,
                                'coordinates': bcoords},
                    }
                                
        geojson['features'].append(feature)
    return geojson

In [ ]:
level = 1
gdf = gpd.read_file(f"mapdata/AdministrativeBoundariesBARMMMunicipalities20190206PSA2016.shp", encoding='utf-8')
gdf.head()

In [ ]:
geojsdata = shapefile_to_geojson(gdf, list(gdf.index))

In [ ]:
trace = go.Choroplethmapbox(z=bm['high'],
                            locations = bm['Mun_Code'], 
                            colorscale = 'Greens',
                            colorbar = dict(thickness=20, ticklen=3),
                            geojson = geojsdata,
                            text = bm['Mun_Name'],
                            hovertemplate = '<b>State</b>: <b>%{text}</b>'+
                                            '<br> <b>Val </b>: %{z}<br>',
                            marker_line_width=0.1, marker_opacity=0.7)
layout = go.Layout(title_text ='Norway mapbox choropleth', title_x =0.5, width=750, height=700,
                   mapbox = dict(center= dict(lat=121.7740, lon=12.8797),            
                                 accesstoken= mapboxt,
                                 zoom=3
                               ))

fig=go.Figure(layout=layout)
fig.show()

In [ ]:
bm

In [ ]:
mapboxt='pk.eyJ1Ijoia2V2c2VzdHJlbGxhIiwiYSI6ImNrNWlwcWpvZDBoNGEza21zeDc0OWczeDIifQ.-ajWL8TrUDMCN1OzzXTjhg'

In [ ]:
bm

In [ ]:
bm.to_file("path_to_GeoJSON _file", driver = "GeoJSON")
with open("path_to_GeoJSON _file") as geofile:
    j_file = json.load(geofile)

In [ ]:
for feature in j_file["features"]:
    feature['id']= feature['properties']['Mun_Code']

In [ ]:
import plotly.express as px

In [ ]:
fig = px.choropleth_mapbox(pd.DataFrame(bm)[['Mun_Code','high']], geojson=j_file, locations='Mun_Code',
                           color='high', color_continuous_scale='Viridis', range_color=(0,12),
                           mapbox_style="carto-positron", zoom=3, center = {"lat": 7.8797, "lon": 124.7740},
                           opacity=0.5,labels={'high':'risk'})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()